### 1️⃣ 安装 SDK
运行下方单元格以安装 PageIndex OCR SDK。该操作仅需在你的环境中执行一次。

In [ ]:
!pip install --upgrade pageindex

### 2️⃣ 初始化客户端

导入 PageIndex 客户端类，并使用你的 API 密钥完成身份验证。请务必妥善保管你的 API 密钥，切勿公开分享。

In [ ]:
from pageindex import PageIndexClient

# Paste your API key here, you can get the api key from https://dash.pageindex.ai/api-keys
API_KEY = "API_KEY"
pi_client = PageIndexClient(api_key=API_KEY)

### 3️⃣ 提交 PDF 文档以进行 OCR 识别

使用客户端上传 PDF 文件执行 OCR 识别处理（当前仅支持 PDF 文件格式）。

提交完成后，你会获取一个doc_id，可用于查询任务状态以及获取 OCR 识别结果。

> 如需替换文件，请将下方的文件路径修改为你本地的 PDF 文件路径。

In [ ]:
import requests, os

pdf_url = "https://arxiv.org/pdf/2501.12948.pdf"
pdf_path = os.path.join("../data", pdf_url.split('/')[-1])
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

response = requests.get(pdf_url)
with open(pdf_path, "wb") as f:
    f.write(response.content)

print(f"Downloaded file to: {pdf_path}")
result = pi_client.submit_document(pdf_path)
doc_id = result["doc_id"]
print(f"Document submitted. Document ID: {doc_id}")

Downloaded file to: ../data/2501.12948.pdf
Document submitted. Document ID: pi-cmdx77gn0003108poki6h7cpc


### 4️⃣ 查询处理状态并获取 OCR 识别结果

OCR 识别处理耗时因文件大小而异：小文件仅需数秒，大文件则可能需要数分钟。

以下代码会每隔 3 秒轮询一次服务状态，最长轮询时长为 5 分钟。待处理完成后，代码会预览提取出的第一页文本内容。

In [ ]:
import time

# Simple polling
for attempt in range(60):  # Try up to 300s (100 x 3s)
    ocr_result = pi_client.get_ocr(doc_id)
    if ocr_result["status"] == "completed":
        print("OCR Results ready!")
        break
    elif ocr_result["status"] == "failed":
        print("OCR failed.")
        break
    time.sleep(3)
else:
    print("Still processing after 10 minutes. Try again later.")

# Preview the first page's markdown
if ocr_result.get("status") == "completed":
    if ocr_result["result"]:
        first_page = ocr_result["result"][0]
        print(f"Page {first_page['page_index']} (partial content):\n")
        print(first_page["markdown"][:1000])  # Print first 1000 chars
    else:
        print("No pages found in OCR result.")
else:
    print("OCR not completed yet. Try again later.")

OCR Results ready!
Page 1 (partial content):

# DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning

DeepSeek-AI<br>research@deepseek.com


## Abstract

We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without supervised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities. Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing reasoning behaviors. However, it encounters challenges such as poor readability, and language mixing. To address these issues and further enhance reasoning performance, we introduce DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeekR1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models (1.5B, 7B,

In [ ]:
def extract_toc_from_json(data, indent_size=2):
    headings = []

    def extract_from_nodes(nodes, parent_level=0):
        if isinstance(nodes, list):
            for node in nodes:
                if isinstance(node, dict):
                    # 先处理节点自身的标题
                    if 'title' in node:
                        page_idx = node.get('page_index', 0)
                        headings.append((parent_level + 1, node['title'], page_idx, len(headings)))

                    # 再处理子节点
                    if 'nodes' in node:
                        extract_from_nodes(node['nodes'], parent_level + 1)
        elif isinstance(nodes, dict):
            # 先处理节点自身的标题
            if 'title' in nodes:
                page_idx = nodes.get('page_index', 0)
                headings.append((parent_level + 1, nodes['title'], page_idx, len(headings)))

            # 再处理子节点
            if 'nodes' in nodes:
                extract_from_nodes(nodes['nodes'], parent_level + 1)

    if isinstance(data, dict) and 'pages' in data:
        for page in data['pages']:
            if 'markdown' in page:
                page_idx = page.get('page_index', 0)
                extracted = _extract_headings_from_markdown(page['markdown'], page_idx)
                for level, title, _ in extracted:
                    headings.append((level, title, page_idx, len(headings)))
    else:
        extract_from_nodes(data, -1)  # 从-1开始，这样第一级标题的level是0

    toc_lines = []
    for level, title, page_index, order in headings:
        indent = '  ' * level
        toc_lines.append(f"{indent}- {title}")

    return '\n'.join(toc_lines)

def _extract_headings_from_markdown(markdown_content, page_index=0):
    import re
    headings = []
    heading_pattern = r'^(#{1,6})\s+(.+)$'

    for line in markdown_content.split('\n'):
        match = re.match(heading_pattern, line.strip())
        if match:
            level = len(match.group(1))
            title = re.sub(r'\s+', ' ', match.group(2).strip())
            if title:
                headings.append((level, title, page_index))

    return headings

### 5️⃣ 获取文档树形结构

你也可以通过下方方法获取该文档的 PageIndex 树形结构。若树形结构暂未生成，请稍后重试。

In [ ]:
from pprint import pprint

def remove_text_fields(data):
    if isinstance(data, dict):
        return {k: remove_text_fields(v) for k, v in data.items() if k != 'text'}
    elif isinstance(data, list):
        return [remove_text_fields(item) for item in data]
    return data

tree_result = pi_client.get_tree(doc_id)
# print the tree structure
if tree_result.get("status") == "completed":
    print("Document tree structure loaded!")
    toc = extract_toc_from_json(tree_result.get("result"))

    print("\n## Tree Structure (drop text fields)\n")
    pprint(remove_text_fields(tree_result.get("result")),sort_dicts=False)
else:
    print(f"Tree status: {tree_result.get('status')}. Try again later if still processing.")

Document tree structure loaded!

## Tree Structure (drop text fields)

[{'title': 'DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via '
           'Reinforcement Learning',
  'node_id': '0000',
  'page_index': 1,
  'nodes': [{'title': 'Abstract', 'node_id': '0001', 'page_index': 1},
            {'title': 'Contents', 'node_id': '0002', 'page_index': 2},
            {'title': '1. Introduction',
             'node_id': '0003',
             'page_index': 3,
             'nodes': [{'title': '1.1. Contributions',
                        'node_id': '0004',
                        'page_index': 4},
                       {'title': '1.2. Summary of Evaluation Results',
                        'node_id': '0005',
                        'page_index': 4}]},
            {'title': '2. Approach',
             'node_id': '0006',
             'page_index': 5,
             'nodes': [{'title': '2.1. Overview',
                        'node_id': '0007',
                        'page_index': 5},
 

In [ ]:
if tree_result.get("status") == "completed":
    print("Document tree structure loaded!")
    toc = extract_toc_from_json(tree_result.get("result"))
    print(toc)
else:
    print(f"Tree status: {tree_result.get('status')}. Try again later if still processing.")

Document tree structure loaded!
- DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning
  - Abstract
  - Contents
  - 1. Introduction
    - 1.1. Contributions
    - 1.2. Summary of Evaluation Results
  - 2. Approach
    - 2.1. Overview
    - 2.2. DeepSeek-R1-Zero: Reinforcement Learning on the Base Model
      - 2.2.1. Reinforcement Learning Algorithm
      - 2.2.2. Reward Modeling
      - 2.2.3. Training Template
      - 2.2.4. Performance, Self-evolution Process and Aha Moment of DeepSeek-R1-Zero
    - 2.3. DeepSeek-R1: Reinforcement Learning with Cold Start
      - 2.3.1. Cold Start
      - 2.3.2. Reasoning-oriented Reinforcement Learning
      - 2.3.3. Rejection Sampling and Supervised Fine-Tuning
      - 2.3.4. Reinforcement Learning for all Scenarios
    - 2.4. Distillation: Empower Small Models with Reasoning Capability
  - 3. Experiment
    - 3.1. DeepSeek-R1 Evaluation
    - 3.2. Distilled Model Evaluation
  - 4. Discussion
    - 4.1. Distillation 

### 6️⃣ 删除文档（清理操作）

若你不再需要该文档，可运行下方代码将其删除。

In [ ]:
pi_client.delete_document(doc_id)
print("Document deleted successfully.")

## 💬 注意事项

- 目前仅支持PDF 文件格式。